<h1>Operartis-1A KNN Algorithm</h1>

In [27]:
import pandas as pd
import numpy as np
import os 

In [28]:
filename = os.path.join(os.getcwd(), "train.csv")
df = pd.read_csv(filename, header=0)

In [29]:
df.head()


,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,...,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation,generatorAllocation
0,1287,9/18/13,MANUAL,Jane Doe,0,A,1291.0,9/18/13,NONE,5481.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/11/13_3446886485_600 cc AAB
1,1293,9/18/13,MANUAL,Jane Doe,0,A,1313.0,9/18/13,NONE,"34,233.97",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_211 AAB
2,1293,9/18/13,MANUAL,Jane Doe,0,A,1314.0,9/18/13,NONE,"32,643.27",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_209 AAB
3,1293,9/18/13,MANUAL,Jane Doe,0,A,1315.0,9/18/13,NONE,"36,897.54",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_205 AAB
4,1293,9/18/13,MANUAL,Jane Doe,0,A,1316.0,9/18/13,NONE,"31,444.64",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULT,USD_9/17/13_3445996130_202 AAB


In [30]:
df.shape

(223937, 31)

<h2>Splitting the DataFrames into two seprate dataframes one for Transaction A and one for Transaction B</h2>

In [31]:
#List of all columns in the dataframe 
name_of_columns = df.columns.tolist()
print(name_of_columns)

['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'A_debitOrCredit', 'A_amount', 'A_valueDate', 'A_currencyCode', 'A_account', 'A_transactionReferences', 'A_transactionAttributes', 'A_orderingPartyInfo', 'A_receivingPartyInfo', 'generatedArtifactName', 'generatedId', 'generatedImportDate', 'B_debitOrCredit', 'B_amount', 'B_valueDate', 'B_currencyCode', 'B_account', 'B_transactionReferences', 'B_transactionAttributes', 'B_orderingPartyInfo', 'B_receivingPartyInfo', 'targetAllocation', 'generatorAllocation']


In [32]:
df = df[df['targetAllocation'] != 'MULT']

In [33]:
columns_A = ['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'A_debitOrCredit', 'A_amount', 'A_valueDate', 'A_currencyCode', 'A_account', 'A_transactionReferences', 'A_transactionAttributes', 'A_orderingPartyInfo', 'A_receivingPartyInfo', 'generatorAllocation']
columns_B = ['matchId', 'matchDate', 'matchRule', 'matchedBy', 'wasPreviouslyMismatched', 'generatorArtifactName', 'generatorId', 'generatorImportDate', 'B_debitOrCredit', 'B_amount', 'B_valueDate', 'B_currencyCode', 'B_account', 'B_transactionReferences', 'B_transactionAttributes', 'B_orderingPartyInfo', 'B_receivingPartyInfo', 'targetAllocation']

In [34]:
df_A = df[columns_A]
df_A.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,generatorAllocation
18,1296,9/18/13,MANUAL,Jane Doe,0,A,1337.0,9/18/13,NONE,"7,184.36",9/6/13,USD,3446205303,NaN,936 AAB,NaN,NaN,USD_9/6/13_3446205303_936 AAB
26,1304,9/18/13,MANUAL,Jane Doe,0,A,1368.0,9/18/13,NONE,"66,875.79",9/16/13,USD,3445812411,NaN,1833 AAB,NaN,NaN,USD_9/16/13_3445812411_1833 AAB
58,1317,9/18/13,MANUAL,Jane Doe,0,A,1418.0,9/18/13,NONE,"68,015.10",9/12/13,USD,3445989615,NaN,668 AAB,NaN,NaN,USD_9/12/13_3445989615_668 AAB
84,1325,9/18/13,MANUAL,Jane Doe,0,A,1461.0,9/18/13,NONE,5555.14,9/5/13,USD,3446264408,NaN,784 AAB,NaN,NaN,USD_9/5/13_3446264408_784 AAB
117,1338,9/18/13,MANUAL,Jane Doe,0,A,1519.0,9/18/13,NONE,"46,805.07",9/13/13,USD,3445812411,NaN,1832 AAB,NaN,NaN,USD_9/13/13_3445812411_1832 AAB


In [35]:
df_B = df[columns_B]
df_B.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation
18,1296,9/18/13,MANUAL,Jane Doe,0,A,1337.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD_9/6/13_3446205303_936 AAB
26,1304,9/18/13,MANUAL,Jane Doe,0,A,1368.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD_9/16/13_3445812411_1833 AAB
58,1317,9/18/13,MANUAL,Jane Doe,0,A,1418.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD_9/12/13_3445989615_668 AAB
84,1325,9/18/13,MANUAL,Jane Doe,0,A,1461.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD_9/5/13_3446264408_784 AAB
117,1338,9/18/13,MANUAL,Jane Doe,0,A,1519.0,9/18/13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USD_9/13/13_3445812411_1832 AAB


In [36]:
df_A.shape


(172653, 18)

In [37]:
df_A = df_A[df_A['A_amount'].notna()]

df_A.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,generatorAllocation
18,1296,9/18/13,MANUAL,Jane Doe,0,A,1337.0,9/18/13,NONE,"7,184.36",9/6/13,USD,3446205303,NaN,936 AAB,NaN,NaN,USD_9/6/13_3446205303_936 AAB
26,1304,9/18/13,MANUAL,Jane Doe,0,A,1368.0,9/18/13,NONE,"66,875.79",9/16/13,USD,3445812411,NaN,1833 AAB,NaN,NaN,USD_9/16/13_3445812411_1833 AAB
58,1317,9/18/13,MANUAL,Jane Doe,0,A,1418.0,9/18/13,NONE,"68,015.10",9/12/13,USD,3445989615,NaN,668 AAB,NaN,NaN,USD_9/12/13_3445989615_668 AAB
84,1325,9/18/13,MANUAL,Jane Doe,0,A,1461.0,9/18/13,NONE,5555.14,9/5/13,USD,3446264408,NaN,784 AAB,NaN,NaN,USD_9/5/13_3446264408_784 AAB
117,1338,9/18/13,MANUAL,Jane Doe,0,A,1519.0,9/18/13,NONE,"46,805.07",9/13/13,USD,3445812411,NaN,1832 AAB,NaN,NaN,USD_9/13/13_3445812411_1832 AAB


In [38]:
df_B = df_B[df_B['B_amount'].notna()]

df_B.head()

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation
345,3,9/20/13,:000001,Jane Doe,0,A,3.0,9/18/13,NONE,"-2,310.84",9/12/13,USD,3.446051e+09,NaN,868.0,NaN,NaN,USD_9/13/13_3446050625_BOA Tra AAA
346,4,9/20/13,:000001,Jane Doe,0,A,4.0,9/18/13,NONE,"-4,518.81",9/2/13,USD,3.446205e+09,NaN,868.0,NaN,NaN,USD_9/6/13_3446205303_BOA Tra AAA
347,5,9/20/13,:000001,Jane Doe,0,A,5.0,9/18/13,NONE,"-1,502.62",9/5/13,USD,3.446854e+09,NaN,868.0,NaN,NaN,USD_9/6/13_3446853861_BOA Tra AAA
348,6,9/20/13,:000001,Jane Doe,0,A,6.0,9/18/13,NONE,5298.47,9/5/13,USD,9.286037e+08,NaN,868.0,NaN,NaN,USD_9/6/13_928603651_BOA Tra AAA
349,8,9/20/13,:000001,Jane Doe,0,A,8.0,10/3/13,NONE,"-134,445.48",9/19/13,USD,3.446051e+09,NaN,868.0,NaN,NaN,USD_9/20/13_3446050638_BOA Tra AAA


In [39]:
print(df_A.shape)
print(df_B.shape)

(162064, 18)
(170196, 18)


#There are more A Transaction info then B transaction information

# Check for duplicates and remove them if they exist.

In [40]:
duplicate_rows_A = df_A[df_A.duplicated()]
duplicate_rows_A

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,generatorAllocation


In [41]:
duplicate_rows_B = df_B[df_B.duplicated()]
duplicate_rows_B

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation
9190,13713,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5529.35,10/11/13,USD,3.445812e+09,NaN,165.0,NaN,NaN,USD_10/11/13_3445812260_403 AAB
10118,19253,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5384.25,10/2/13,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/2/13_3451320936_401 AAB
10123,19253,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5679.74,10/2/13,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/2/13_3451320936_401 AAB
10125,19253,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5384.25,10/2/13,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/2/13_3451320936_401 AAB
10126,19253,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5384.25,10/2/13,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/2/13_3451320936_401 AAB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212676,2310451,8/27/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,6320.78,8/25/16,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_8/25/16_3451320936_409 AAB
215947,2313575,9/16/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5343.19,9/15/16,USD,3.445812e+09,NaN,399.0,NaN,NaN,USD_8/15/16_3445812260_ICB/Recl AAA
223676,2343699,10/29/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,6176.51,10/28/16,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/28/16_3451320936_413 AAB
223677,2343699,10/29/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,6176.51,10/28/16,USD,3.451321e+09,NaN,165.0,NaN,NaN,USD_10/28/16_3451320936_413 AAB


In [42]:
df_B = df_B.drop_duplicates()
duplicate_rows_B = df_B[df_B.duplicated()]
duplicate_rows_B

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation


# Check all B transactions have targetAllocation and all A transactions have generatorAllocation

In [43]:
df_A[df_A['generatorAllocation'].isnull()]

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,A_debitOrCredit,A_amount,A_valueDate,A_currencyCode,A_account,A_transactionReferences,A_transactionAttributes,A_orderingPartyInfo,A_receivingPartyInfo,generatorAllocation


In [44]:
df_B[df_B['targetAllocation'].isnull()]

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation
3085,6598,9/20/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5377.19,9/5/13,USD,3.445812e+09,NaN,399.0,NaN,NaN,NaN
5190,6598,10/16/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5309.19,8/23/13,USD,3.445812e+09,NaN,695.0,NaN,NaN,NaN
8323,11747,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5309.83,10/11/13,USD,3.445812e+09,NaN,695.0,NaN,NaN,NaN
8324,11747,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5376.55,10/15/13,USD,3.445812e+09,NaN,172.0,NaN,NaN,NaN
9197,13718,10/28/13,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,4864.12,10/18/13,USD,3.445812e+09,NaN,555.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222778,2376068,10/25/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5135.11,10/22/16,USD,4.121359e+09,NaN,555.0,NaN,NaN,NaN
223328,2335957,10/28/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5420.08,10/27/16,USD,4.121359e+09,NaN,165.0,NaN,NaN,NaN
223329,2335957,10/28/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,5266.3,10/27/16,USD,4.121359e+09,NaN,455.0,NaN,NaN,NaN
223913,2347324,11/1/16,MANUAL,Jane Doe,0,NaN,NaN,NaN,NONE,"4,343.19",10/29/16,USD,3.451567e+09,NaN,699.0,NaN,NaN,NaN


In [45]:
df_B = df_B.dropna(subset=['targetAllocation'])
df_B[df_B['targetAllocation'].isnull()]

,matchId,matchDate,matchRule,matchedBy,wasPreviouslyMismatched,generatorArtifactName,generatorId,generatorImportDate,B_debitOrCredit,B_amount,B_valueDate,B_currencyCode,B_account,B_transactionReferences,B_transactionAttributes,B_orderingPartyInfo,B_receivingPartyInfo,targetAllocation


<h2> Cleaning and preprocessing the data for Model </h2>

In [51]:
columns_to_checkA = ['A_amount', 'A_valueDate', 'A_account']
columns_to_checkB = ['B_amount', 'B_valueDate', 'B_account']

# Filter rows with 'NONE' in any of the specified columns in dataset A
filtered_rows_A = df_A[df_A[columns_to_checkA].apply(lambda row: 'NONE' in row.values, axis=1)]

# Filter rows with 'NONE' in any of the specified columns in dataset B
filtered_rows_B = df_B[df_B[columns_to_checkB].apply(lambda row: 'NONE' in row.values, axis=1)]

# Count the number of rows containing 'NONE' in each dataset
count_A = len(filtered_rows_A)
count_B = len(filtered_rows_B)

print(f"Number of rows containing 'NONE' in dataset A: {count_A}")
print(f"Number of rows containing 'NONE' in dataset B: {count_B}")

Number of rows containing 'NONE' in dataset A: 10
Number of rows containing 'NONE' in dataset B: 0


In [54]:
#Dropping rows containing none for A
df_A = df_A.drop(filtered_rows_A.index)

In [57]:
# Convert 'A_amount' to float


df_A['A_amount'] = df_A['A_amount'].astype(str)

# Replace commas in 'A_amount'
df_A['A_amount'] = df_A['A_amount'].str.replace(',', '', regex=True)

# Convert 'A_amount' to numeric, handling non-numeric values
df_A['A_amount'] = pd.to_numeric(df_A['A_amount'], errors='coerce')

data_type_AmountA = df_A['A_amount'].dtypes

print(data_type_AmountA)


float64


In [60]:
# Convert 'B_amount' to float

df_B['B_amount'] = df_B['B_amount'].astype(str)

# Replace commas in 'B_amount'
df_B['B_amount'] = df_B['B_amount'].str.replace(',', '', regex=True)

# Convert 'A_amount' to numeric, handling non-numeric values
df_B['B_amount'] = pd.to_numeric(df_B['B_amount'], errors='coerce')

data_type_AmountB = df_B['B_amount'].dtypes

print(data_type_AmountA)

float64


<h2> Nearest Neighbours Model </h2>

In [61]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Create copies of the data to avoid modifying the original DataFrames
data_A_copy = df_A[['A_amount', 'A_account', 'A_valueDate']].copy()
data_B_copy = df_B[['B_amount', 'B_account', 'B_valueDate']].copy()

# Fit the scaler on Transaction A data
data_A_scaled = scaler.fit_transform(data_A_copy)

# Use the same scaler to transform Transaction B data
data_B_scaled = scaler.transform(data_B_copy)

ValueError: could not convert string to float: '9/6/13'